<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_21_Keypad_Conundrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [319]:
input1 = """029A
980A
179A
456A
379A"""

In [320]:
input = """539A
964A
803A
149A
789A"""

In [321]:
from collections import deque

def get_direction(from_pos, to_pos):
    """Get the direction command to move from one position to another"""
    from_row, from_col = from_pos
    to_row, to_col = to_pos

    if from_row > to_row:
        return '^' * (from_row - to_row)  # Multiple steps up
    if from_row < to_row:
        return 'v' * (to_row - from_row)  # Multiple steps down
    if from_col > to_col:
        return '<' * (from_col - to_col)  # Multiple steps left
    if from_col < to_col:
        return '>' * (to_col - from_col)  # Multiple steps right
    return ''

def find_path_multi(sequence, keypad):
    """Process a sequence of buttons and return possible paths of commands"""
    gap_pos = keypad['X']
    current_pos = keypad['A']
    all_paths = {''}  # Start with empty path

    for char in sequence.strip():
        if char not in keypad or char == 'X':
            continue

        target_pos = keypad[char]
        curr_row, curr_col = current_pos
        target_row, target_col = target_pos

        # Check if each path crosses the gap
        vert_crosses_gap = (curr_col == gap_pos[1] and
                           min(curr_row, target_row) <= gap_pos[0] <= max(curr_row, target_row))
        horz_crosses_gap = (curr_row == gap_pos[0] and
                           min(curr_col, target_col) <= gap_pos[1] <= max(curr_col, target_col))

        new_paths = set()
        for base_path in all_paths:
            # Add vertical-first path if valid
            if not vert_crosses_gap:
                vert_path = base_path
                if curr_row != target_row:
                    vert_path += get_direction(current_pos, (target_row, curr_col))
                if curr_col != target_col:
                    vert_path += get_direction((target_row, curr_col), target_pos)
                vert_path += 'A'
                new_paths.add(vert_path)

            # Add horizontal-first path if valid
            if not horz_crosses_gap:
                horz_path = base_path
                if curr_col != target_col:
                    horz_path += get_direction(current_pos, (curr_row, target_col))
                if curr_row != target_row:
                    horz_path += get_direction((curr_row, target_col), target_pos)
                horz_path += 'A'
                new_paths.add(horz_path)

        all_paths = new_paths
        current_pos = target_pos

    return all_paths

num_keypad = {
    '7': (0, 0), '8': (0, 1), '9': (0, 2),
    '4': (1, 0), '5': (1, 1), '6': (1, 2),
    '1': (2, 0), '2': (2, 1), '3': (2, 2),
    'X': (3, 0), '0': (3, 1), 'A': (3, 2)
}
dir_keypad = {
    'X': (0, 0), '^': (0, 1), 'A': (0, 2),
    '<': (1, 0), 'v': (1, 1), '>': (1, 2),
}
total = 0
for x in input.split():
  n = int(x[:-1])
  # print(x)
  x = find_path_multi(x, num_keypad)
  # print(x)
  x = [s for si in x for s in find_path_multi(si, dir_keypad)]
  # print(x)
  x = [s for si in x for s in find_path_multi(si, dir_keypad)]
  # print(x)
  print(min(len(s) for s in x))
  total += min(len(s) for s in x) * n
total

70
72
76
76
66


231564

In [324]:
transitions = {
    # From A to other buttons
    "A to ^": {"A to <", "< to A"},
    "A to <": {"A to v", "v to <", "< to <", "< to A"},
    "A to v": {"A to <", "< to v", "v to A"},
    "A to >": {"A to v", "v to A"},
    "A to A": {"A to A"},

    # From ^ to other buttons
    "^ to A": {"A to >", "> to A"},
    "^ to <": {"A to v", "v to <", "< to A"},
    "^ to v": {"A to v", "v to A"},
    "^ to >": {"A to v", "v to >", "> to A"},
    "^ to ^": {"^ to ^"},

    # From < to other buttons
    "< to A": {"A to >", "> to >", "> to ^", "^ to A"},
    "< to ^": {"A to >", "> to ^", "^ to A"},
    "< to v": {"A to >", "> to A"},
    "< to >": {"A to >", "> to >", "> to A"},
    "< to <": {"< to <"},

    # From v to other buttons
    "v to A": {"A to ^", "^ to >", "> to A"},
    "v to ^": {"A to ^", "^ to A"},
    "v to <": {"A to <", "< to A"},
    "v to >": {"A to >", "> to A"},
    "v to v": {"v to v"},

    # From > to other buttons
    "> to A": {"A to ^", "^ to A"},
    "> to ^": {"A to <", "< to ^", "^ to A"},
    "> to <": {"A to <", "< to <", "< to A"},
    "> to v": {"A to <", "< to A"},
    "> to >": {"> to >"}
}

def count_transitions(path):
    buttons = ['A']  # Start with initial state
    for c in path:
        if c in '^v<>A':
            buttons.append(c)

    transition_counts = {}
    for i in range(len(buttons)-1):
        from_button = buttons[i]
        to_button = buttons[i+1]
        transition = f"{from_button} to {to_button}"
        transition_counts[transition] = transition_counts.get(transition, 0) + 1

    return transition_counts

def translate_path(transition_counts, next_level_map):
    next_level_counts = {}

    # For each high-level transition and its count
    for transition, count in transition_counts.items():
        # Get the set of sub-transitions needed
        if transition in next_level_map:
            sub_transitions = next_level_map[transition]
            # Add count to each sub-transition
            for sub_transition in sub_transitions:
                next_level_counts[sub_transition] = (
                    next_level_counts.get(sub_transition, 0) + count
                )

    return next_level_counts

total = 0
for code in input.split():
  n = int(code[:-1])
  paths = find_path_multi(code, num_keypad)
  cs = [count_transitions(x) for x in paths]
  for i in range(25):
    cs = [translate_path(x, transitions) for x in cs]
  min_len = min([sum(x.values()) for x in cs])
  total += min_len * n
total

281212077733592